# Population Demographics of Countries Using CIA Factbook Data

In this project, I will be quickly examining data from the [CIA World Factbook](https://www.cia.gov/library/publications/the-world-factbook/), which contains statistics about every country. First, let's connect to the data (which can be downloaded [here](https://dsserver-prod-resources-1.s3.amazonaws.com/257/factbook.db)) and take a quick look at the dataset.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

In [2]:
%%sql
SELECT *
  FROM facts
 LIMIT 5

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


## 1. Minimum and Maximum Population Values

Let's take a look at the maximum and minimum population and the maximum and minimum population growth for this dataset, which we can observe by the query below.

In [3]:
%%sql
SELECT MIN(population) as "Min Pop",
       MAX(population) as "Max Pop",
       MIN(population_growth) as "Min Pop Growth",
       MAX(population_growth) as "Max Pop Growth"
  FROM facts

 * sqlite:///factbook.db
Done.


Min Pop,Max Pop,Min Pop Growth,Max Pop Growth
0,7256490011,0.0,4.02


It looks as though the minimum and maximum population values are inaccurate, as there are no countries in the world with as few as zero people and also none with as many as 7 billion people.

### a) Editing the Max and Min Population Values

Let's take a look at the data entries with the min and max population values to see why these values seem to differ significantly from common knowledge. First, let's take a look at the min value.

In [4]:
%%sql
SELECT name as "Location"
  FROM facts
 WHERE population == (SELECT MIN(population)
                        FROM facts)

 * sqlite:///factbook.db
Done.


Location
Antarctica


Antartica is in this dataset even though it is not a country. I will have to remove it from our dataset prior to analyzing the rest of the data. Let's now take a look at the max population value.

In [5]:
%%sql
SELECT name as "Location"
  FROM facts
 WHERE population == (SELECT MAX(population)
                        FROM facts)

 * sqlite:///factbook.db
Done.


Location
World


The world is also not a country, so this can also be removed from the dataset. Let's calculate the min/max population and min/max population growth once again after removing these two values from our dataset.

In [6]:
%%sql
SELECT MIN(population) as "Min Pop",
       MAX(population) as "Max Pop",
       MIN(population_growth) as "Min Pop Growth",
       MAX(population_growth) as "Max Pop Growth"
  FROM facts
 WHERE name <> "World"
    AND name <> "Antarctica"

 * sqlite:///factbook.db
Done.


Min Pop,Max Pop,Min Pop Growth,Max Pop Growth
48,1367485388,0.0,4.02


In [7]:
%%sql
SELECT MIN(population) as "Min Pop", 
       name as Location
  FROM facts
 WHERE name <> "Antarctica"

 * sqlite:///factbook.db
Done.


Min Pop,Location
48,Pitcairn Islands


### b) Re-evaluating the Data

These values for min/max population make much more sense than the previous values, since China has a population for around 1.3 billion and the Pitcairn Islands is a small territory owned by the British Isles with a very scarce population. Note that this dataset will be a bit inaccurate as it might contain categories other than countries. Now I will take a look at all locations with more than 100 million people.

In [8]:
%%sql
SELECT name location,
       population
  FROM facts
 WHERE population > 100000000
 ORDER BY population desc

 * sqlite:///factbook.db
Done.


location,population
World,7256490011
China,1367485388
India,1251695584
European Union,513949445
United States,321368864
Indonesia,255993674
Brazil,204259812
Pakistan,199085847
Nigeria,181562056
Bangladesh,168957745


Note that the European Union is not a country, but rather a collection of countries. When analyzing the rest of the data, I will remove the European Union, the World, and Antartica from the database. There are probably other countries (and territories) that could be removed as well, but for now I will just remove those three.

## 2. Futher Analysis

Now that I have a better understanding about the data, including which values I should exclude from the data, I can begin to analyze the data.

### a) Average Population and Area

First, I will calculate the average population and average area of all countries in the database.

In [9]:
%%sql
SELECT ROUND(AVG(population), 2) AS "Average Population",
       ROUND(AVG(area), 2) AS "Average Area (km2)"
  FROM facts
 WHERE name <> "World"
   AND name <> "Antarctica"
   AND name <> "European Union"

 * sqlite:///factbook.db
Done.


Average Population,Average Area (km2)
30362063.59,539893.19


In [10]:
%%sql
SELECT ROUND(AVG(population), 2) AS "Average Population",
       ROUND(AVG(area), 2) AS "Average Area (km2)"
  FROM facts
 WHERE name <> "World" AND name <> "Antarctica" AND name <> "European Union"
   AND population <> "None" AND area <> "None"

 * sqlite:///factbook.db
Done.


Average Population,Average Area (km2)
30622240.58,567094.63


The average country has around 30 million people and is slightly larger than 550k square kilometers.

### b) Countries with Both Large Populations and Small Areas

Now that I have calculated the average population and average area of each country, I want to find all countries that have a population that is larger than average as well as all countries that have an area that is smaller than average.

In [11]:
%%sql
SELECT name "Country", 
       population "Population", 
       area "Area (km2)"
  FROM facts
 WHERE population > (SELECT AVG(population)
                       FROM facts
                      WHERE (name <> "World" AND name <> "Antarctica" AND name <> "European Union")
                        AND (population <> "None" AND area <> "None")
                    )
   AND area < (SELECT AVG(area)
                 FROM facts
                WHERE (name <> "World" AND name <> "Antarctica" AND name <> "European Union")
                  AND (population <> "None" AND area <> "None")
              )
  
 ORDER BY population DESC

 * sqlite:///factbook.db
Done.


Country,Population,Area (km2)
Bangladesh,168957745,148460
Japan,126919659,377915
Philippines,100998376,300000
Vietnam,94348835,331210
Germany,80854408,357022
Thailand,67976405,513120
United Kingdom,64088222,243610
Italy,61855120,301340
"Korea, South",49115196,99720
Spain,48146134,505370


It looks as though there are only 15 countries that fit this description, many of which are known for having a dense population.

### c) Population Density

I am curious about which countries have the greast population density as well as which countries have the smallest population density. Population density is the number of people that live per square kilometer (on average), so I will create this measure using the population and area columns in the dataset. First, I will look at the top ten most densely populated countries.

In [12]:
%%sql
SELECT name "Region", 
       population "Population", 
       area "Area (km2)",
       ROUND(CAST(population as float) / area, 2) "Population Density"
  FROM facts
 WHERE name <> "World" AND name <> "Antarctica" AND name <> "European Union"
   AND "Population Density" <> "None"
 ORDER BY "Population Density" DESC
 LIMIT 10

 * sqlite:///factbook.db
Done.


Region,Population,Area (km2),Population Density
Macau,592731,28,21168.96
Monaco,30535,2,15267.5
Singapore,5674472,697,8141.28
Hong Kong,7141106,1108,6445.04
Gaza Strip,1869055,360,5191.82
Gibraltar,29258,6,4876.33
Bahrain,1346613,760,1771.86
Maldives,393253,298,1319.64
Malta,413965,316,1310.02
Bermuda,70196,54,1299.93


As expected, all of these countries are incredibly small and size and many are city-states. Let's take a look at the least densely populated countries as well.

In [13]:
%%sql
SELECT name "Region", 
       population "Population", 
       area "Area (km2)",
       ROUND(CAST(population as float) / area, 2) "Population Density"
  FROM facts
 WHERE name <> "World" AND name <> "Antarctica" AND name <> "European Union"
   AND "Population Density" <> "None"
 ORDER BY "Population Density"
 LIMIT 10

 * sqlite:///factbook.db
Done.


Region,Population,Area (km2),Population Density
Greenland,57733,2166086,0.03
Svalbard,1872,62045,0.03
Falkland Islands (Islas Malvinas),3361,12173,0.28
Pitcairn Islands,48,47,1.02
Mongolia,2992908,1564116,1.91
Western Sahara,570866,266000,2.15
Namibia,2212307,824292,2.68
Australia,22751014,7741220,2.94
Iceland,331918,103000,3.22
Guyana,735222,214969,3.42


As expected, most of the countries on this list are remote islands, four of which (including the top three least dense regions) are very far north. The four countries that aren't remote island are surprising, although it is noteworthy that all of them are poor, with Namibia being the wealthiest of the four at around $6000 GDP per capita.